<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/framework/framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!mkdir /framework
!mkdir /framework/dev
!mkdir /framework/dev/project
!mkdir /framework/dev/project/conf/
!mkdir /framework/dev/project/bin/
!mkdir /framework/dev/project/models/
!mkdir /framework/dev/project/lib/
!mkdir /framework/dev/project/lib/utils
!mkdir /framework/dev/project/lib/db
!touch /framework/dev/project/bin/sourceFileCopy.py
!chmod 775 /framework/dev/project/bin/sourceFileCopy.py
!mkdir /framework/dev/project/data/
!mkdir /framework/dev/project/data/in/
!mkdir /framework/dev/project/data/out/
!mkdir /framework/dev/project/data/processing/
!mkdir /framework/dev/project/data/reject/
!echo "Hello World" > /framework/dev/project/data/in/testsourcefile.csv

mkdir: cannot create directory ‘/framework’: File exists
mkdir: cannot create directory ‘/framework/dev’: File exists
mkdir: cannot create directory ‘/framework/dev/project’: File exists
mkdir: cannot create directory ‘/framework/dev/project/conf/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/bin/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/models/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/lib/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/lib/utils’: File exists
mkdir: cannot create directory ‘/framework/dev/project/lib/db’: File exists
mkdir: cannot create directory ‘/framework/dev/project/data/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/data/in/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/data/out/’: File exists
mkdir: cannot create directory ‘/framework/dev/project/data/processing/’: File exists
mkdir: cannot create directory ‘/framewo

In [4]:
%%writefile /framework/dev/frameWork.ksh
if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/dev/'
fi 

if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/qa/'
fi 

Writing /framework/dev/frameWork.ksh


In [5]:
%%writefile /framework/dev/project/conf/project.ksh
if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/dev/project'
fi 

if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/qa/project'
fi 


Writing /framework/dev/project/conf/project.ksh


In [6]:
%%writefile /framework/dev/project/bin/genExecute.ksh
#!/bin/ksh

#check for Environment and running profiles
dirName=`dirname $0`

if [[ $dirName = *[dD][eE][vV]* ]]
  then
  export environment="DEV"
  echo $environment
elif [[ $dirName = *[qQ][aQ]* ]]
  then
  export environment="QA"
  echo $environment
else
  export environment="UNKNOWN"
  echo $environment
  exit 6
fi 

CONFPATH=`echo $dirName|sed -e 's/bin/conf/g'`
if [ ! -f "${CONFPATH}/project.ksh" ];then
  echo "${CONFPATH}/project.ksh is missing"
  exit 7
fi
echo "sourcing ${CONFPATH}/project.ksh"
. ${CONFPATH}/project.ksh $environment 2>/dev/null 1>/dev/null

CONFPATH=`echo $dirName|sed -e 's/project\/bin//g'`
if [ ! -f "${CONFPATH}/frameWork.ksh" ];then
  echo "${CONFPATH}/frameWork.ksh is missing"
  exit 7
fi
echo "sourcing ${CONFPATH}/frameWork.ksh"
. ${CONFPATH}/frameWork.ksh $environment 2>/dev/null 1>/dev/null

echo $(dirname $0)/$*
$(dirname $0)/$*


Writing /framework/dev/project/bin/genExecute.ksh


In [29]:
%%writefile /framework/dev/project/conf/sourceFileCopyConf.json
{
    "1":{
        "step-types":{
            "FILE_COPY":[{
                "templateName":"SOURCE_FILE_COPY",
                "SOURCE":{
                    "Parameters":[{
                        "SOURCE_PATH":"/framework/dev/project/data/in/",
                        "SOURCE_FILE":"testsourcefile.csv"
                    }]
                },
                "TARGET":{
                    "Parameters":[{
                        "TARGET_PATH":"/framework/dev/project/data/processing/",
                        "APPEND_TIMESTAMP":"True"
                    }]
                }
            }]
        }
    }
}

Overwriting /framework/dev/project/conf/sourceFileCopyConf.json


In [33]:
%%writefile /framework/dev/project/bin/sourceFileCopy.py
#!/usr/local/bin/python
#generic script to caopy the source data
#Name           : sourceFileCopy.py
#Descritpion    : script copies/move file from source directory
#
#/../../sourceFileCopy.ksh -cfg sourceFileCopyConfig.json
#History
#Version      | Date        | Editor      |Comment
#-------------------------------------------------------------------------
#1            | 28/07/2021  | Sumith      | initial version
#
import os
import sys
import argparse
import logging
import signal
import threading
import time
from os import path
import json

lib_path = os.path.abspath(os.path.join(__file__,'..','..','lib'))
sys.path.append(lib_path)

class sourceFileCopyBase(object):
  """base Class for task SourceFileCopy
  """
  sleep_timeout = 1
    
  def __init__(self, app_name):
    """Constructor.
    """
    self._log = logging.getLogger('')
    self._app_name = app_name
    self._stop_event = threading.Event()

  def __del__(self):
    pass

  def _on_start(self):
    """Subclass to implement what needs to run dring start of process
    """
    pass

  def _on_stop(self):
    """Subclass to implement what needs to run dring start of process
    """
    pass      

  def _install_signal_handlers(self):
    """Install signal handler for the process.
    """
    self._log.debug('Installing Signal handler')   

    def handler(signum, _):
      """signal handler.
      """
      self._log.warning('Got Signal %s', signum)
      self._on_stop()
      sys.exit(signum)   

      for sig in (signal.SIGHUP, signal.SIGINT, signal.SIGTERM,
                  signal.SIGQUIT, signal.SIGABRT):
        signal.signal(sig, handler)

  def readConfigFile(self, configFile):
    if path.exists(configFile):
      self._log.info('reading config file')
      with open(configFile) as configFile:
        configData = json.load(configFile)
      self._log.debug('configData : %s', json.dumps(configData, indent=4))
    return configData

  def _main_loop(self, configData):
    """Main Loop
    """
    for key in range(1, 99):
      try:
        (configData[str(key)])
      except KeyError:
        self._log.info('No More steps')
        break

  def run(self, configFile):
    """Run application
    """
    self.configFile = configFile
    
    self._log.info('HOSTNAME : %s', os.uname()[1])
    self._log.info('USERID : %s', os.environ.get('USER'))
    self._log.info('SCRIPT PATH : %s', os.path.abspath(__file__))
    self._log.info('ConfigFile : %s', self.configFile)

    exit_status = 0
    self._install_signal_handlers()

    try:
      self._on_start()
      configData = self.readConfigFile(configFile)
      self._main_loop(configData)
    except BaseException as exc:
      self._log.exception('Unrecoverable exception in %s main loop.'
                          'Exiting: %s', self._app_name, exc)
      exit_status = 1
    finally:
      self._on_stop()
      self._log.info('Done.')
    return exit_status
    

def main(config):

  class sourceFileCopy(sourceFileCopyBase):
    """initializing base call
    """

  if (config.debug):
    logging.basicConfig(format='%(asctime)s:%(filename)s[%(lineno)d]:%(process)d:%(levelname)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.DEBUG)
  else:
    logging.basicConfig(format='%(asctime)s:%(filename)s[%(lineno)d]:%(process)d:%(levelname)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

  configFile = config.configfile
  returnCode = sourceFileCopy('SourceFileCopy').run(configFile)
  if returnCode != 0:
    sys.exit(1)


if __name__ == '__main__':
  parser = argparse.ArgumentParser(description='''Description : Source File copy ''',
                                   formatter_class=argparse.RawDescriptionHelpFormatter,
                                   epilog='''USAGE; {0} -cfg [config] '''.format(os.path.basename(sys.argv[0])))
  
  parser.add_argument('--configfile', '-c',
                      help='Config json file',
                      required=True)
  
  parser.add_argument('--debug', '-v',
                      action='store_true',
                      help='Debug/Verbose Mode')
  
  config = parser.parse_args(sys.argv[1:])

  main(config)

Overwriting /framework/dev/project/bin/sourceFileCopy.py


In [34]:
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json
07/28/2021 04:15:14 PM:sourceFileCopy.py[89]:646:INFO: HOSTNAME : 205e1b142eea
07/28/2021 04:15:14 PM:sourceFileCopy.py[90]:646:INFO: USERID : None
07/28/2021 04:15:14 PM:sourceFileCopy.py[91]:646:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
07/28/2021 04:15:14 PM:sourceFileCopy.py[92]:646:INFO: ConfigFile : /framework/dev/project/conf/sourceFileCopyConf.json
07/28/2021 04:15:14 PM:sourceFileCopy.py[68]:646:INFO: reading config file
07/28/2021 04:15:14 PM:sourceFileCopy.py[81]:646:INFO: No More steps
07/28/2021 04:15:14 PM:sourceFileCopy.py[107]:646:INFO: Done.


In [35]:
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json --debug

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json --debug
07/28/2021 04:15:16 PM:sourceFileCopy.py[89]:666:INFO: HOSTNAME : 205e1b142eea
07/28/2021 04:15:16 PM:sourceFileCopy.py[90]:666:INFO: USERID : None
07/28/2021 04:15:16 PM:sourceFileCopy.py[91]:666:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
07/28/2021 04:15:16 PM:sourceFileCopy.py[92]:666:INFO: ConfigFile : /framework/dev/project/conf/sourceFileCopyConf.json
07/28/2021 04:15:16 PM:sourceFileCopy.py[53]:666:DEBUG: Installing Signal handler
07/28/2021 04:15:16 PM:sourceFileCopy.py[68]:666:INFO: reading config file
07/28/2021 04:15:16 PM:sourceFileCopy.py[71]:666:DEBUG: configData : {
    "1": {
        "step-types": {
            "FILE_COPY": [
                {
                    "templateName": "SOURCE_FILE_COPY",
                    "SOURCE": {
                

In [12]:
!cat /framework/dev/project/data/in/testsourcefile.csv

Hello World


In [24]:
ls -rlt /framework/dev/project/conf/sourceFileCopyConf.json

-rw-r--r-- 1 root root 616 Jul 28 16:01 /framework/dev/project/conf/sourceFileCopyConf.json
